In [1]:
from data_preprocess import *
from data_visual_tools import *
import pandas as pd
import os
from PIL import Image, ImageDraw, ImageFile
from matplotlib import pyplot as plt
import shutil

In [17]:
# Set the directory path
dir_path = '/home/ec2-user/segmenter/MAIDA/data1000/images'
out_dir = '/home/ec2-user/segmenter/ETTDATA/MIMIC'

i = 0
# Loop through each file in the directory
for filename in os.listdir(dir_path):
    if filename.endswith('.png'):
        # Open the image file using PIL
        img = Image.open(os.path.join(dir_path, filename))
        
        # Set the output file name and path
        output_filename = os.path.splitext(filename)[0] + ".jpg"
        output_path = os.path.join(out_dir, output_filename)
        
        # Convert and save the image as JPEG
        img.save(output_path, "JPEG")
        
        i += 1
        if i % 50 == 0:
            print(f"Converted {i} images")


Converted 50 images
Converted 100 images
Converted 150 images
Converted 200 images
Converted 250 images
Converted 300 images
Converted 350 images
Converted 400 images
Converted 450 images
Converted 500 images
Converted 550 images
Converted 600 images
Converted 650 images
Converted 700 images
Converted 750 images
Converted 800 images
Converted 850 images
Converted 900 images
Converted 950 images
Converted 1000 images
Converted 1050 images
Converted 1100 images


In [20]:
input_dir = '/home/ec2-user/efs/RANZCR/data'
dst_dir = '/home/ec2-user/segmenter/ETTDATA/RANZCR'

i = 0
for src_dir in os.listdir(input_dir):
    print(f"Processing {src_dir}")
    if not src_dir.startswith("."):
        file_list = os.listdir(os.path.join(input_dir, src_dir))

        for file_name in file_list:
            src_file = os.path.join(input_dir, src_dir, file_name)
            dst_file = os.path.join(dst_dir, file_name)

            shutil.copy(src_file, dst_file)
            i += 1
            if i % 50 == 0:
                print(f"Copied {i} files.")
    


Processing abnormal_u
Copied 50 files.
Processing borderline_u
Copied 100 files.
Copied 150 files.
Copied 200 files.
Copied 250 files.
Copied 300 files.
Copied 350 files.
Copied 400 files.
Processing .DS_Store
Processing normal_u
Copied 450 files.
Copied 500 files.
Copied 550 files.
Copied 600 files.
Copied 650 files.
Copied 700 files.
Copied 750 files.
Copied 800 files.
Processing no_ETT_u
Copied 850 files.
Copied 900 files.


In [22]:
def train_val_split(data_dir, target_dir, train_info, val_info):
    for file in os.listdir(data_dir):
        # print(file)
        name = file[:-4]
        if name in train_info:
            shutil.copy(os.path.join(data_dir, file), os.path.join(target_dir, "train", file))
        elif name in val_info:
            shutil.copy(os.path.join(data_dir, file), os.path.join(target_dir, "val", file))


In [23]:
def get_split_info(group, split_info='/home/ec2-user/segmenter/ETT_Evaluation/split/train_mimic_only.csv'):
    split = pd.read_csv(split_info)
    if group == 'train':
        files = split[split['Split']=='train']
    elif group == 'val':
        files = split[split['Split']=='val']
    elif group == 'test':
        files = split[split['Split']=='test']
    else:
        raise ValueError("Invalid group.")
    
    m = list(files[files['Source']=='MIMIC']['FileName'])
    r = list(files[files['Source']=='RANZCR']['FileName'])
    return m, r

In [24]:
def split_images(split_info, mimic_data_dir, ranzcr_data_dir, output_dir):
    train_m, train_r = get_split_info('train', split_info=split_info)
    val_m, val_r = get_split_info('val', split_info=split_info)
    train_val_split(mimic_data_dir, output_dir, train_m, val_m)
    train_val_split(ranzcr_data_dir, output_dir, train_r, val_r)
    

In [26]:
split_images('/home/ec2-user/segmenter/ETT_Evaluation/data_split/sampled_data_split.csv', 
             '/home/ec2-user/segmenter/ETTDATA/MIMIC',
             '/home/ec2-user/segmenter/ETTDATA/RANZCR',
             '/home/ec2-user/segmenter/ETTDATA/sampled_data_split')

In [31]:
def create_test_folder(split_info, data_dir_m, data_dir_r, target_dir_m, target_dir_r):
    test_m, test_r = get_split_info('test', split_info=split_info)
    for file in os.listdir(data_dir_m):
        name = file[:-4]
        if name in test_m:
            shutil.copy(os.path.join(data_dir_m, file), os.path.join(target_dir_m, file))
    
    for file in os.listdir(data_dir_r):
        name = file[:-4]
        if name in test_r:
            shutil.copy(os.path.join(data_dir_r, file), os.path.join(target_dir_r, file))


In [32]:
create_test_folder('/home/ec2-user/segmenter/ETT_Evaluation/data_split/all_data_split.csv',
                   '/home/ec2-user/segmenter/ETTDATA/MIMIC',
                   '/home/ec2-user/segmenter/ETTDATA/RANZCR',
                   '/home/ec2-user/segmenter/ETTDATA/Test/MIMIC',
                   '/home/ec2-user/segmenter/ETTDATA/Test/RANZCR')